# Using Climada on the Euler Cluster (ETH internal)

### Content

1. [Access to Euler](#ssh)
2. [Installation and working directories](#dir)
3. [Pre-installed version of Climada](#pic)
    1. [Load dependencies](#lmod)
    2. [Check installation](#check)
    3. [Adjust the Climada configuration](#conf)
    4. [Run a job](#bsub)
4. [Working with Git Branches](#branch)
    1. [Load dependencies](#lmod2)
    2. [Create installation environment](#venv)
    3. [Check out sources](#clone)
    4. [Pip install Climada](#pip)
    5. [Check installation](#check2)
    6. [Adjust the Climada configuration](#conf2)
    7. [Run a job](#bsub2)
5. [Fallback: Conda installation](#fallb)
    1. [Conda installation](#conda)
    4. [Check out sources](#git)
    3. [Climada environemnt](#env)
    5. [Adjust the Climada configuration](#conf3)
    6. [Climada sripts](#bash)
    7. [Run a job](#bsub3)
6. [Conda Deinstallation](#Rm)
    1. [Conda](#RmConda)
    2. [Climada](#RmClimada)

## <a id="ssh">Access to Euler</a>

See https://scicomp.ethz.ch/wiki/Getting_started_with_clusters for details on how to register at and get started with Euler.

For all steps below, first enter the Cluster via SSH.

## <a id="dir">Installation- and working directories</a>

Please, get familiar with the various Euler storage options:  https://scicomp.ethz.ch/wiki/Storage_systems.<br>
As a general rule: use `/cluster/project` for installation and `/cluster/work` for data processing.

For ETH WCR group members, the suggested installation and working directories are `/cluster/project/climate/$USER` and `/cluster/work/climate/$USER` respectively.<br>
You may have to create the installation directory:

```bash
mkdir -p /cluster/project/climate/$USER \
         /cluster/work/climate/$USER
```

## <a id="pic">Pre-installed version of Climada</a>

Climada is pre-installed and available in the default pip environment of Euler.

### <a id="lmod">1. Load dependencies</a>
```bash
env2lmod
module load gcc/8.2.0 python/3.9.9 gdal/3.4.3 geos/3.9.1 proj/8.2.1 libspatialindex/1.9.3 netcdf/4.6.3 eccodes/2.22.1 zlib/1.2.9 libtiff/4.2.0 sqlite/3.35.5
```

You need to execute these two lines every time you login to Euler before Climada can be used. To safe yourself from doing it manually, one can append these lines to the ~/.bashrc script, which is automatically executed upon logging in to Euler.

### <a id="check">2. Check installation</a>
```bash
python -c 'import climada; print(climada.__file__)'
```
 should output something like this:
```bash
/cluster/apps/nss/gcc-8.2.0/python/3.9.9/x86_64/lib64/python3.9/site-packages/climada/__init__.py
```

### <a id="conf">3. Adjust the Climada configuration</a>
Edit a configuration file according to your needs (see [Guide_Configuration](../guide/Guide_Configuration.ipynb)).
Create a climada.conf file e.g., in /cluster/home/$USER/.config with the following content:

```json
{
    "local_data": {
        "system": "/cluster/work/climate/USERNAME/climada/data",
        "demo": "/cluster/project/climate/USERNAME/climada_python/data/demo",
        "save_dir": "/cluster/work/climate/USERNAME/climada/results"
    }
}
```
(Replace USERNAME with your nethz-id.)

### <a id="bsub">4. Run a job</a>
Please see the docs at https://slurm.schedmd.com/ on how to use the `slurm` batch system 
and the Wiki https://scicomp.ethz.ch/wiki/Transition_from_LSF_to_Slurm for a mapping of `lsf` commands to their `slurm` equivalents.

```bash
cd /cluster/work/climate/$USER  # change to the working directory
sbatch [slurm options*] --wrap 'python climada_job_script.py'  # submit the job
```

## <a id="branch">Working with Git branches</a>
If the Climada version of the default installation is not according to your needs, you can install Climada from a local Git repository.

### <a id="lmod2">1. Load dependencies</a>
See [Load dependencies](#lmod) above.

### <a id="venv">2. Create installation environment</a>
```bash
python -m venv --system-site-packages /cluster/project/climate/$USER/climada_venv
```

### <a id="clone">3. Checkout sources</a>
```bash
cd /cluster/project/climate/$USER
git clone https://github.com/CLIMADA-project/climada_python.git
cd climada_python
git checkout develop  # i.e., your branch of interest
```

### <a id="pip">4. Pip install Climada</a>
```bash
source /cluster/project/climate/$USER/climada_venv/bin/activate
pip install -e /cluster/project/climate/$USER/climada_python
```

### <a id="check2">5. Check installation</a>
```bash
cd /cluster/work/climate/$USER
python -c 'import climada; print(climada.__file__)'
```

should output exactly this (with explicit \$USER):
```bash
/cluster/project/climate/$USER/climada_python/climada/__init__.py
```

### <a id="conf2">6. Adjust the Climada configuration</a>
See [Adjust the Climada configuration](#conf) above.

### <a id="bsub2">7. Run a job</a>
See [Run a job](#bsub) above.

## <a id="fallb">Fallback: Conda installation</a>
If Climada cannot be installed through pip because of changed dependency requirements, there is still the possibility to install Climada through the Conda environment.
> **WARNING:** This approach is highly discouraged, as it imposes a heavy and mostly unnecessary burden on the file system of the cluster.

### 1. <a id="conda"> Conda Installation </a>

Download or update to the latest version of [Miniconda](https://conda.io/miniconda.html).<br>
Installation is done by execution of the following steps:

```bash
cd /cluster/project/climate/USERNAME
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh
miniconda3/bin/conda init
rm Miniconda3-latest-Linux-x86_64.sh
```

During the installation process of Miniconda, you are prompted to set the working directory according to your choice.
Set it to `/cluster/project/climate/USERNAME/miniconda3`.<br>
Once the installation has finished, log out of Euler and in again. The command prompt should be preceded by `(base)`, 
indicating that the installation was a success and that you login in into conda's base environment by default.

### 2. <a id="git"> Checkout sources </a>
See [Checkout sources](#git) above.

###  3. <a id="env"> Climada Environment </a>

Create the conda environment:

```bash
cd /cluster/project/climate/USERNAME/climada_python
conda env create -f requirements/env_climada.yml --name climada_env
conda env update -n climada_env -f requirements/env_developer.yml

conda activate climada_env
conda install conda-build
conda develop .
```

### 4. <a id="conf3"> Adjust the Climada configuration </a>

See [Adjust the Climada configuration](#conf) above.

###  5. <a id="bash"> Climada Scripts </a>

Create a bash script for executing python scripts in the climada environment, `climadajob.sh`:
```bash
#!/bin/bash
PYTHON_SCRIPT=$1
shift
. ~/.bashrc
conda activate climada_env
python $PYTHON_SCRIPT $@
echo $PYTHON_SCRIPT completed
```

Make it executable:

```bash
chmod +x climadajob.sh
```

Create a python script that executes climada code, e.g., `climada_smoke_test.py`:

```python
import sys
from climada import CONFIG, SYSTEM_DIR
from climada.util.test.test_finance import TestNetpresValue
TestNetpresValue().test_net_pres_val_pass()
print(SYSTEM_DIR)
print(CONFIG.local_data.save_dir.str())
print("the script ran with arguments", sys.argv)
```

### 6. <a id="bsub3"> Run a Job </a>

With the scripts from above you can submit the python script as a job like this:

```bash
sbatch [slurm options] --wrap "/path/to/climadajob.sh /path/to/climada_smoke_test.py arg1 arg2"
```

After the job has finished the slurm output file should look something like this:

```txt
/cluster/work/climate/USERNAME/climada/data
/cluster/work/climate/USERNAME/climada/results
the script ran with arguments ['/path/to/climada_smoke_test.py', 'arg1' 'arg2']
python_script.sh completed
```

Please see the docs at https://slurm.schedmd.com/ on how to use the `slurm` batch system 
and the Wiki https://scicomp.ethz.ch/wiki/Transition_from_LSF_to_Slurm for a mapping of `lsf` commands to their `slurm` equivalents.

## <a id="Rm">Conda Deinstallation</a>

### 1. <a id="RmConda"> Conda </a>

Remove the miniconda3 directory from the installation directory:

```bash
rm -rf /cluster/project/climate/USERNAME/miniconda3/
```

Delete the conda related parts from `/cluster/home/USERNAME/.bashrc`, i.e., everything between

`# >>> conda initialize >>>`\
and\
`# <<< conda initialize <<<`

### 2. <a id="RmClimada"> Climada </a>

Remove the climada sources and config file:

```bash
rm -rf /cluster/project/climate/USERNAME/climada_python
rm -f /cluster/home/USERNAME/climada.conf /cluster/home/USERNAME/*/climada.conf
```